In [1]:
%pip install firebase_admin

Note: you may need to restart the kernel to use updated packages.


In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

import pandas as pd

from helper import compare, select, pprint, order

In [3]:
# firebase_admin.delete_app(firebase_admin.get_app())

In [4]:
METROS = 'metros'

In [5]:
cred = credentials.Certificate("./project-d247d-firebase-adminsdk-hpskj-b7566ae1c5.json")

default_app = firebase_admin.initialize_app(cred, {
	'databaseURL': 'https://project-d247d-default-rtdb.firebaseio.com/',
})

In [6]:
METROS_REF = db.reference(METROS).get()
print(METROS_REF[0])

{'metro': 'Green Bay', 'population': 320050, 'region': 'WI', 'teams': ['Green Bay Packers', 'Green Bay Phoenix', 'Green Bay Phoenix', 'Green Bay Phoenix', 'Green Bay Phoenix']}


In [7]:
# SELECT
COLS = ['metro', 'population']

res = [select(obj, COLS) for obj in METROS_REF]
pprint(res)

       metro  population
0  Green Bay      320050
1     Sarnia       96151
2   Hamilton      747545
3     Regina      236481
4   Edmonton     1321426


In [8]:
# FILTER
FIELD = 'population'
OPERATOR = '>='
VALUE = '320051'

res = [obj for obj in METROS_REF if compare(obj[FIELD], OPERATOR, VALUE)]
pprint(res)

      metro  population region  \
0  Hamilton      747545     ON   
1  Edmonton     1321426     AB   
2  Winnipeg      778489     MB   
3    Ottawa     1323783     ON   
4  Montreal     4098927     QC   

                                               teams  
0  [Hamilton Alerts, Hamilton Flying Wildcats, Ha...  
1  [Edmonton Elks, Edmonton Eskimos, Edmonton Oil...  
2  [Winnipeg ‘Pegs, Winnipeg Blue Bombers, Winnip...  
3  [Ottawa Redblacks, Ottaw Renegades, Ottawa Rou...  
4  [Montreal Expos, Montreal AAA Winged Wheelers,...  


In [9]:
# GROUPING/AGGREGATION
FIELD = 'region'
AGGS = ['count', 'sum', 'avg', 'min', 'max']
AGG_COL = 'population'

groups = {}
for obj in METROS_REF:
    if obj[FIELD] in groups:
        groups[obj[FIELD]].append(obj)
    else:
        groups[obj[FIELD]] = [obj]

aggs = []
for region_group, metro in groups.items():
    obj = {}
    obj[FIELD] = region_group

    obj['min'], obj['max'] = float('inf'), float('-inf')
    obj['sum'] = 0
    for metro_area in metro:
        val = metro_area[AGG_COL]
        obj['min'] = min(val, obj['min'])
        obj['max'] = max(val, obj['max'])
        obj['sum'] += val
    obj['count'] = len(metro)
    obj['avg'] = obj['sum'] / len(metro)

    aggs.append(obj)

pprint(aggs)

  region      min      max      sum  count           avg
0     WI        0  1576236  2802998      9  3.114442e+05
1     ON    96151  5928040  8095519      4  2.023880e+06
2     SK   236481   236481   236481      1  2.364810e+05
3     AB  1321426  1392609  2714035      2  1.357018e+06
4     MB   778489   778489   778489      1  7.784890e+05


In [10]:
FIELD = 'population'
OPERATOR = 'asc'

a = [(obj[FIELD], obj) for obj in METROS_REF]
res = order(a, OPERATOR)
pprint(res)

                 metro  population region  \
0  New York Metro Area    20320876          
1  Greater Los Angeles    13353907     CA   
2              Chicago     9533040     IL   
3    Dallas-Fort Worth     7399662     TX   
4              Houston     6892427     TX   

                                               teams  
0  [Brooklyn Dodgers, Brooklyn Robins, Brooklyn S...  
1  [Anaheim Angels, California Angels, Los Angele...  
2  [Chicago Cubs, Chicago White Sox, Chicago Bull...  
3  [Texas Rangers, Dallas Mavericks, Dallas Cowbo...  
4  [Houston Astros, Houston Colt .45's, Houston R...  
